# Deploy a model to a batch endpoint
Imagine you trained a model to predict the product sales. The model has been trained and tracked in Azure Machine Learning. Every month, you want to use the model to forecast the sales for the upcoming month.

In many production scenarios, long-running tasks that deal with large amounts of data are performed as batch operations. In machine learning, batch inferencing is used to asynchronously apply a predictive model to multiple cases and write the results to a file or database.

In Azure Machine Learning, you can implement batch inferencing solutions by deploying a model to a batch endpoint.

## Understand and create batch endpoints
To get a model to generate batch predictions, you can deploy the model to a batch endpoint.

You'll learn how to use batch endpoints for asynchronous batch scoring.

## Batch predictions
To get batch predictions, you can deploy a model to an endpoint. An endpoint is an HTTPS endpoint that you can call to trigger a batch scoring job. The advantage of such an endpoint is that you can trigger the batch scoring job from another service, such as Azure Synapse Analytics or Azure Databricks. A batch endpoint allows you to integrate the batch scoring with an existing data ingestion and transformation pipeline.

Whenever the endpoint is invoked, a batch scoring job is submitted to the Azure Machine Learning workspace. The job typically uses a compute cluster to score multiple inputs. The results can be stored in a datastore, connected to the Azure Machine Learning workspace.

## Create a batch endpoint
To deploy a model to a batch endpoint, you'll first have to create the batch endpoint.

To create a batch endpoint, you'll use the BatchEndpoint class. Batch endpoint names need to be unique within an Azure region.

To create an endpoint, use the following command:

In [ ]:
# create a batch endpoint
endpoint = BatchEndpoint(
    name="endpoint-example",
    description="A batch endpoint",
)

ml_client.batch_endpoints.begin_create_or_update(endpoint)

>Explore the reference documentation to [create a batch endpoint with the Python SDK v2](https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.batchendpoint).

## Deploy a model to a batch endpoint
You can deploy multiple models to a batch endpoint. Whenever you call the batch endpoint, which triggers a batch scoring job, the default deployment will be used unless specified otherwise.

![alt text](assets/batch-details.png)

## Use compute clusters for batch deployments
The ideal compute to use for batch deployments is the Azure Machine Learning compute cluster. If you want the batch scoring job to process the new data in parallel batches, you need to provision a compute cluster with more than one maximum instances.

To create a compute cluster, you can use the AMLCompute class.


In [ ]:
from azure.ai.ml.entities import AmlCompute

cpu_cluster = AmlCompute(
    name="aml-cluster",
    type="amlcompute",
    size="STANDARD_DS11_V2",
    min_instances=0,
    max_instances=4,
    idle_time_before_scale_down=120,
    tier="Dedicated",
)

cpu_cluster = ml_client.compute.begin_create_or_update(cpu_cluster)

# Deploy your MLflow model to a batch endpoint
An easy way to deploy a model to a batch endpoint is to use an MLflow model. Azure Machine Learning will automatically generate the scoring script and environment for MLflow models.

To deploy an MLflow model, you need to have created an endpoint. Then you can deploy the model to the endpoint.

## Register an MLflow model
To avoid needed a scoring script and environment, an MLflow model needs to be registered in the Azure Machine Learning workspace before you can deploy it to a batch endpoint.

To register an MLflow model, you'll use the Model class, while specifying the model type to be MLFLOW_MODEL. To register the model with the Python SDK, you can use the following code:

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

model_name = 'mlflow-model'
model = ml_client.models.create_or_update(
    Model(name=model_name, path='./model', type=AssetTypes.MLFLOW_MODEL)
)

In this example, we're taking the model files from a local path. The files are all stored in a local folder called model. The folder must include the MLmodel file, which describes how the model can be loaded and used.


## Deploy an MLflow model to an endpoint
To deploy an MLflow model to a batch endpoint, you'll use the BatchDeployment class.

When you deploy a model, you'll need to specify how you want the batch scoring job to behave. The advantage of using a compute cluster to run the scoring script (which is automatically generated by Azure Machine Learning), is that you can run the scoring script on separate instances in parallel.

When you configure the model deployment, you can specify:

- instance_count: Count of compute nodes to use for generating predictions.
- max_concurrency_per_instance: Maximum number of parallel scoring script runs per compute node.
- mini_batch_size: Number of files passed per scoring script run.
- output_action: What to do with the predictions: summary_only or append_row.
- output_file_name: File to which predictions will be appended, if you choose append_row for output_action.

>Explore the reference documentation to [create a batch deployment with the Python SDK v2](https://learn.microsoft.com/en-us/python/api/azure-ai-ml/azure.ai.ml.entities.batchdeployment).

To deploy an MLflow model to a batch endpoint, you can use the following code:


In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="forecast-mlflow",
    description="A sales forecaster",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

# Deploy a custom model to a batch endpoint
If you want to deploy a model to a batch endpoint without using the MLflow model format, you need to create the scoring script and environment.

To deploy a model, you must have already created an endpoint. Then you can deploy the model to the endpoint.

## Create the scoring script
The scoring script is a file that reads the new data, loads the model, and performs the scoring.
The scoring script must include two functions:
- init(): Called once at the beginning of the process, so use for any costly or common preparation like loading the model.
- run(): Called for each mini batch to perform the scoring.

The run() method should return a pandas DataFrame or an array/list.

A scoring script may look as follows:

In [ ]:
import os
import mlflow
import pandas as pd


def init():
    global model

    # get the path to the registered model file and load it
    model_path = os.path.join(os.environ["AZUREML_MODEL_DIR"], "model")
    model = mlflow.pyfunc.load(model_path)


def run(mini_batch):
    print(f"run method start: {__file__}, run({len(mini_batch)} files)")
    resultList = []

    for file_path in mini_batch:
        data = pd.read_csv(file_path)
        pred = model.predict(data)

        df = pd.DataFrame(pred, columns=["predictions"])
        df["file"] = os.path.basename(file_path)
        resultList.extend(df.values)

    return resultList

There are some things to note from the example script:
- AZUREML_MODEL_DIR is an environment variable that you can use to locate the files associated with the model.
- Use global variable to make any assets available that are needed to score the new data, like the loaded model.
- The size of the mini_batch is defined in the deployment configuration. If the files in the mini batch are too large to be processed, you need to split the files into smaller files.
- By default, the predictions will be written to one single file.

>Learn more about how to [author scoring scripts for batch deployments](https://learn.microsoft.com/en-us/azure/machine-learning/batch-inference/how-to-batch-scoring-script).

## Create an environment

Your deployment requires an execution environment in which to run the scoring script. Any dependency your code requires should be included in the environment.

You can create an environment with a Docker image with Conda dependencies, or with a Dockerfile.

You'll also need to add the library azureml-core as it is required for batch deployments to work.

To create an environment using a base Docker image, you can define the Conda dependencies in a conda.yaml file:

```yaml

name: basic-env-cpu
channels:
  - conda-forge
dependencies:
  - python=3.8
  - pandas
  - pip
  - pip:
      - azureml-core
      - mlflow


```

Then, to create the environment, run the following code:

In [ ]:
from azure.ai.ml.entities import Environment

env = Environment(
    image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    conda_file="./src/conda-env.yml",
    name="deployment-environment",
    description="Environment created from a Docker image plus Conda environment.",
)
ml_client.environments.create_or_update(env)

## Configure and create the deployment
Finally, you can configure and create the deployment with the BatchDeployment class.

In [ ]:
from azure.ai.ml.entities import BatchDeployment, BatchRetrySettings
from azure.ai.ml.constants import BatchDeploymentOutputAction

deployment = BatchDeployment(
    name="forecast-mlflow",
    description="A sales forecaster",
    endpoint_name=endpoint.name,
    model=model,
    compute="aml-cluster",
    code_path="./code",
    scoring_script="score.py",
    environment=env,
    instance_count=2,
    max_concurrency_per_instance=2,
    mini_batch_size=2,
    output_action=BatchDeploymentOutputAction.APPEND_ROW,
    output_file_name="predictions.csv",
    retry_settings=BatchRetrySettings(max_retries=3, timeout=300),
    logging_level="info",
)
ml_client.batch_deployments.begin_create_or_update(deployment)

# Invoke and troubleshoot batch endpoints
When you invoke a batch endpoint, you trigger an Azure Machine Learning pipeline job. The job will expect an input parameter pointing to the data set you want to score.

## Trigger the batch scoring job
To prepare data for batch predictions, you can register a folder as a data asset in the Azure Machine Learning workspace.

You can then use the registered data asset as input when invoking the batch endpoint with the Python SDK:


In [ ]:
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes

input = Input(type=AssetTypes.URI_FOLDER, path="azureml:new-data:1")

job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint.name, 
    input=input)

You can monitor the run of the pipeline job in the Azure Machine Learning studio. All jobs that are triggered by invoking the batch endpoint will show in the Jobs tab of the batch endpoint.

![alt text](assets/batch-jobs.png)

The predictions will be stored in the default datastore.

## Troubleshoot a batch scoring job
The batch scoring job runs as a pipeline job. If you want to troubleshoot the pipeline job, you can review its details and the outputs and logs of the pipeline job itself.

![alt text](assets/child-job.png)

If you want to troubleshoot the scoring script, you can select the child job and review its outputs and logs.

Navigate to the Outputs + logs tab. The logs/user/ folder contains three files that will help you troubleshoot:
- job_error.txt: Summarize the errors in your script.
- job_progress_overview.txt: Provides high-level information about the number of mini-batches processed so far.
- job_result.txt: Shows errors in calling the init() and run() function in the scoring script.


![alt text](assets/child-output.png)